# Developing Codebrain

In [2]:
%env NEO4J_URI=bolt://localhost:7687
%env NEO4J_USER=neo4j
%env NEO4J_PASSWORD=12345678

env: NEO4J_URI=bolt://localhost:7687
env: NEO4J_USER=neo4j
env: NEO4J_PASSWORD=12345678


In [15]:
import os
import json
from typing import List, Dict, Any
from uuid import uuid4
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from neo4j import GraphDatabase

# Load environment variables
load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define schema
strategy_schema = ResponseSchema(
    name="strategies",
    description="A dictionary of strategy names mapped to their descriptions for solving the coding problem.",
    example={"Brute Force": "Try every pair", "Dynamic Programming": "Use memoization"}
)
structured_parser = StructuredOutputParser.from_response_schemas([strategy_schema])

class KnowledgeGraph:
    def __init__(self):
        self.driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

    def close(self):
        self.driver.close()

    def add_problem_with_context(self, problem_data, strategies, implementations):
        with self.driver.session() as session:
            pid = str(uuid4())
            session.run(
                """
                MERGE (p:Problem {id: $pid})
                SET p.title = $title,
                    p.description = $desc,
                    p.difficulty = 'Unknown',
                    p.tags = $tags,
                    p.examples = $examples
                """,
                pid=pid,
                title=problem_data['original'],
                desc=problem_data['problem_intro'],
                tags=problem_data.get('likely_categories', []),
                examples=[json.dumps(ex) for ex in problem_data.get('examples', [])]
            )

            for name, desc in strategies.items():
                session.run(
                    """
                    MERGE (s:Strategy {name: $name})
                    SET s.description = $desc
                    WITH s
                    MATCH (p:Problem {id: $pid})
                    MERGE (p)-[:USES_STRATEGY]->(s)
                    """,
                    name=name, desc=desc, pid=pid
                )

            for impl in implementations:
                cid = str(uuid4())
                session.run(
                    """
                    CREATE (c:CodeSnippet {
                        id: $cid,
                        language: 'Python',
                        code: $code,
                        complexity: 'Unknown',
                        walkthrough: $desc
                    })
                    WITH c
                    MATCH (s:Strategy {name: $strategy})
                    MERGE (s)-[:IMPLEMENTED_BY]->(c)
                    """,
                    cid=cid,
                    code=impl['code'],
                    desc=impl['description'],
                    strategy=impl['strategy']
                )

class AgentBase:
    def __init__(self, memory: ConversationBufferMemory = None):
        self.memory = memory or ConversationBufferMemory(memory_key="chat_history")

class SenseAgent(AgentBase):
    def __init__(self, llm: ChatOpenAI, memory=None):
        super().__init__(memory)
        self.llm = llm

    def run(self, problem_statement: str) -> Dict[str, Any]:
        prompt = (
            f"You are an expert coding interview tutor.\n"
            f"Given the following problem:\n{problem_statement}\n"
            f"Generate a JSON output with:\n"
            f"- problem_intro\n- examples (list of input/output)\n"
            f"- likely_categories\n- constraints"
        )
        response = self.llm([HumanMessage(content=prompt)]).content
        try:
            parsed = json.loads(response)
        except Exception:
            parsed = {
                "problem_intro": "This is a classic coding interview problem.",
                "examples": [],
                "likely_categories": [],
                "constraints": []
            }
        parsed["original"] = problem_statement
        parsed["keywords"] = problem_statement.lower().split()
        self.memory.save_context({"input": problem_statement}, {"sense_output": json.dumps(parsed)})
        return parsed

class MemoryAgent(AgentBase):
    def __init__(self, graph: KnowledgeGraph):
        super().__init__()
        self.graph = graph

    def run(self, parsed_data: Dict[str, Any]) -> List[Dict[str, Any]]:
        return []

    def store(self, problem_data: Dict[str, Any], strategies: Dict[str, str], implementations: List[Dict[str, Any]]):
        self.graph.add_problem_with_context(problem_data, strategies, implementations)

class IntellectAgent(AgentBase):
    def __init__(self, llm: ChatOpenAI, memory=None):
        super().__init__(memory)
        self.llm = llm

    def run(self, parsed_data: Dict[str, Any], memory_examples: List[Dict[str, Any]]) -> Dict[str, str]:
        prompt = (
            f"You are an algorithm designer. Given the problem and examples, suggest 2-3 strategies as JSON.\n"
            f"Problem: {parsed_data['original']}\n"
            f"Examples: {json.dumps(parsed_data.get('examples', []))}\n"
            f"Respond as {{'StrategyName': 'Short description'}}."
        )
        response = self.llm([HumanMessage(content=prompt)]).content
        try:
            return json.loads(response)
        except Exception:
            return {
                "Brute Force": "Try all pairs.",
                "Hash Map": "Use dictionary to store complements."
            }

class StrategyActionAgent(AgentBase):
    def __init__(self, strategy_name: str, strategy_desc: str, llm: ChatOpenAI, memory=None):
        super().__init__(memory)
        self.strategy_name = strategy_name
        self.strategy_desc = strategy_desc
        self.llm = llm

    def run(self, parsed_data: Dict[str, Any]) -> Dict[str, Any]:
        prompt = (
            f"You are a Python engineer. Given the problem:\n"
            f"{parsed_data['original']}\n\n"
            f"Write a Python solution using the '{self.strategy_name}' strategy. "
            f"Add comments, time/space complexity, and a walkthrough.\n"
        )
        code = self.llm([HumanMessage(content=prompt)]).content.strip()
        return {
            "strategy": self.strategy_name,
            "description": self.strategy_desc,
            "code": code
        }

class EgoAgent(AgentBase):
    def run(self, parsed_data, strategy_outputs: List[Dict[str, Any]]) -> Dict[str, Any]:
        preferred = next((s for s in strategy_outputs if "DP" in s["strategy"] or "Dynamic" in s["strategy"]), strategy_outputs[0])
        return {
            "problem": parsed_data['original'],
            "intro": parsed_data['problem_intro'],
            "examples": parsed_data['examples'],
            "categories": parsed_data['likely_categories'],
            "implementations": strategy_outputs,
            "best_solution": preferred
        }

class MultiAgentSystem:
    def __init__(self, llm: ChatOpenAI):
        self.memory = ConversationBufferMemory(memory_key="chat_history")
        self.graph = KnowledgeGraph()
        self.sense = SenseAgent(llm, self.memory)
        self.memory_agent = MemoryAgent(self.graph)
        self.intellect = IntellectAgent(llm, self.memory)
        self.ego = EgoAgent(self.memory)
        self.llm = llm

    def solve_problem(self, problem_statement: str) -> Dict[str, Any]:
        parsed = self.sense.run(problem_statement)
        memory_examples = self.memory_agent.run(parsed)
        strategies = self.intellect.run(parsed, memory_examples)

        implementations = []
        for name, desc in strategies.items():
            agent = StrategyActionAgent(name, desc, self.llm, self.memory)
            implementations.append(agent.run(parsed))

        summary = self.ego.run(parsed, implementations)
        self.memory_agent.store(parsed, strategies, implementations)
        return summary

# ---------- CLI ENTRY -----------

def run_interactive(agentome):
    while True:
        problem = input("Enter a Python coding interview problem (or 'exit' to quit):\n> ")
        if problem.lower() in ['exit', 'quit']:
            break
        result = agentome.solve_problem(problem)
        print("\n✅ Best Strategy:\n", result['best_solution']['code'])

def run_batch(agentome, file_path):
    with open(file_path, 'r') as f:
        problems = json.load(f)
    for item in problems:
        print(f"\n=== Solving: {item.get('title', 'Untitled')} ===")
        result = agentome.solve_problem(item['definition'])
        print("✅ Best Strategy:\n", result['best_solution']['code'])

def main(batch_file: str = None):
    print("\n🔍 Welcome to Codebrain: The Thinking Engine for Coding Problems\n")
    if not OPENAI_API_KEY:
        raise EnvironmentError("Please set OPENAI_API_KEY in your .env file.")

    llm = ChatOpenAI(temperature=0.2, openai_api_key=OPENAI_API_KEY)
    agentome = MultiAgentSystem(llm)

    if batch_file:
        run_batch(agentome, batch_file)
    else:
        run_interactive(agentome)

if __name__ == "__main__":
    # Toggle this path to run in batch mode or leave None for interactive
    main(batch_file="problems.json")



🔍 Welcome to Codebrain: The Thinking Engine for Coding Problems


=== Solving: Two Sum ===
✅ Best Strategy:
 ```python
def two_sum(nums, target):
    # Create a dictionary to store the indices of numbers we have seen so far
    num_indices = {}
    
    # Iterate through the array
    for i, num in enumerate(nums):
        # Calculate the difference needed to reach the target
        diff = target - num
        
        # Check if the difference is in the dictionary
        if diff in num_indices:
            # If it is, return the indices of the two numbers
            return [num_indices[diff], i]
        
        # Otherwise, add the current number and its index to the dictionary
        num_indices[num] = i
    
    # If no solution is found, return an empty list
    return []

# Test the function
nums = [2, 7, 11, 15]
target = 9
print(two_sum(nums, target))  # Output: [0, 1]
```

**Walkthrough:**
1. We create a dictionary `num_indices` to store the indices of numbers we have seen